# AML Final
Melissa Stone Rogers, July 24, 2024

## Introduction
Professional project using Jupyter Lab, python, and needed frameworks to create a data story and pipelined models using the autompg dataset
Commands were used on a Mac machine running zsh.  

### Import and Read Data

In [2]:
import pandas as pd
autompg = pd.read_csv("auto-mpg.csv")
autompg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [5]:
print(autompg.head(n=10))
print(autompg.describe())

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  15.0          8         350.0        165    3693          11.5          70   
2  18.0          8         318.0        150    3436          11.0          70   
3  16.0          8         304.0        150    3433          12.0          70   
4  17.0          8         302.0        140    3449          10.5          70   
5  15.0          8         429.0        198    4341          10.0          70   
6  14.0          8         454.0        220    4354           9.0          70   
7  14.0          8         440.0        215    4312           8.5          70   
8  14.0          8         455.0        225    4425          10.0          70   
9  15.0          8         390.0        190    3850           8.5          70   

   origin                   car name  
0       1  chevrolet chevelle malibu  
1       1          buick skyla

### Train/Test Data Split 

In [9]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(autompg,
                        test_size=0.2, random_state=123)
print('Train size: ', len(train_set), 'Test size: ', len(test_set))

Train size:  318 Test size:  80


### Train and Evaluate a Linear Regression Model 

In [14]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X = train_set[['weight', 'displacement']]
y = train_set['mpg']

X_test = test_set[['weight', 'displacement']]
y_test = test_set['mpg']

lr_model = LinearRegression()
lr_model.fit(X,y)

y_pred = lr_model.predict(X)
print('Results for linear regression on training data')
print('  Default settings')
print('Internal parameters:')
print('   Bias is ', lr_model.intercept_)
print('   Coefficients', lr_model.coef_)
print('   Score', lr_model.score(X,y))
print('MAE is  ', mean_absolute_error(y, y_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y, y_pred)))
print('MSE is ', mean_squared_error(y, y_pred))
print('R^2    ', r2_score(y,y_pred))

y_test_pred = lr_model.predict(X_test)
print()
print('Results for linear regression on test data')
print('MAE is  ', mean_absolute_error(y_test, y_test_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y_test,
y_test_pred)))
print('MSE is ', mean_squared_error(y_test, y_test_pred))
print('R^2    ', r2_score(y_test,y_test_pred))

Results for linear regression on training data
  Default settings
Internal parameters:
   Bias is  43.621433976443285
   Coefficients [-0.00556012 -0.01801839]
   Score 0.7024820582511935
MAE is   3.2290037032091603
RMSE is  4.2789696537676765
MSE is  18.309581297864668
R^2     0.7024820582511935

Results for linear regression on test data
MAE is   3.4980689722029554
RMSE is  4.343131313113012
MSE is  18.862789602942755
R^2     0.6765950182605451


### Pipeline Models

#### Pipeline 1

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

#Define the pipline
autompg_pipe = Pipeline([
    ('median_transform', SimpleImputer(strategy='median')),
    ('scale_transform', StandardScaler()),
    ('lin_reg', LinearRegression())])

# Fit the pipeline
autompg_pipe.fit(X,y)

# Output the intercept and coefficients 
print("The stage bias is " ,
      autompg_pipe.named_steps['lin_reg'].intercept_)
print("The stage feature coefficients are ",
      autompg_pipe.named_steps['lin_reg'].coef_)

# Predict and evaluate on training data
y_pred = autompg_pipe.predict(X)
print('Results for linear regression on training data with pipeline')
print('MAE is  ', mean_absolute_error(y, y_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y, y_pred)))
print('MSE is ', mean_squared_error(y, y_pred))
print('R^2    ', r2_score(y, y_pred))

# Predict and evaluate on test data
y_test_pred = autompg_pipe.predict(X_test)
print('Results for linear regression on test data with pipeline')
print('MAE is  ', mean_absolute_error(y_test, y_test_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y_test, y_test_pred)))
print('MSE is ', mean_squared_error(y_test, y_test_pred))
print('R^2    ', r2_score(y_test, y_test_pred))

The stage bias is  23.412578616352203
The stage feature coefficients are  [-4.76109404 -1.88955493]
Results for linear regression on training data with pipeline
MAE is   3.229003703209161
RMSE is  4.2789696537676765
MSE is  18.309581297864668
R^2     0.7024820582511935
Results for linear regression on test data with pipeline
MAE is   3.4980689722029568
RMSE is  4.3431313131130125
MSE is  18.862789602942758
R^2     0.676595018260545


#### Pipeline 2

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
power = 3
poly_process = PolynomialFeatures(degree=power, include_bias=False)

#Define the pipeline
autompg_pipe = Pipeline([
    ('median_transform', SimpleImputer(strategy='median')), 
    ('poly_process', PolynomialFeatures()),
    ('scale_transform', StandardScaler()),
    ('lin_reg', LinearRegression())])

# Fit the pipeline
autompg_pipe.fit(X,y)

# Output the intercept and coefficients 
print("The stage bias is " ,
      autompg_pipe.named_steps['lin_reg'].intercept_)
print("The stage feature coefficients are ",
      autompg_pipe.named_steps['lin_reg'].coef_)

# Predict and evaluate on training data
y_pred = autompg_pipe.predict(X)
print('Results for linear regression with polynomial features on training data with pipeline')
print('MAE is  ', mean_absolute_error(y, y_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y, y_pred)))
print('MSE is ', mean_squared_error(y, y_pred))
print('R^2    ', r2_score(y, y_pred))

# Predict and evaluate on test data
y_test_pred = autompg_pipe.predict(X_test)
print('Results for linear regression with polynomial features on test data with pipeline')
print('MAE is  ', mean_absolute_error(y_test, y_test_pred))
print('RMSE is ', np.sqrt(mean_squared_error(y_test, y_test_pred)))
print('MSE is ', mean_squared_error(y_test, y_test_pred))
print('R^2    ', r2_score(y_test, y_test_pred))

The stage bias is  23.412578616352206
The stage feature coefficients are  [  0.          -1.49502986 -14.1635106  -11.16752447  24.49126934
  -4.53461007]
Results for linear regression with polynomial features on training data with pipeline
MAE is   2.9597527898224114
RMSE is  4.053977013800068
MSE is  16.43472962841932
R^2     0.7329470918695618
Results for linear regression with polynomial features on test data with pipeline
MAE is   3.1805704976155433
RMSE is  4.284941459858176
MSE is  18.360723314411516
R^2     0.6852030100948552


# Results
Basic results for models to predict mpg on
the auto-mpg data.
| Model | Training Features | Set | RMSE | R2 |
|:---|:---|:---|:---|:---|
|Linear Regression|Weight,Displacement|Training|4.28|70.25|
|Linear Regression|Weight,Displacement|Test|4.34|67.67|
|Pipeline, Linear Regression|Weight,Displacement|Training|4.28|70.25|
|Pipeline, Linear Regression|Weight,Displacement|Test|4.34|67.67|
|Pipeline with Polynomial Features, Linear Regression|Weight,Displacement|Training|4.05|73.29|
|Pipeline with Polynomial Features, Linear Regression|Weight,Displacement|Test|4.28|68.52|


#### Discussion of Results
The performance of the models improved with each iteration, the pipeline model with the polynomial features performing the best. However, none of the models performed spectacularly- none of the R^2 scores were close to 80%. I do not have a concern of over- or under-fitting on any of the models, as the test models vs the training models performed as expected. I even ran the polynomial pipeline with a higher power and saw no discernable difference in model performance. I think my chosen features are not the best features to predict the target feature, mpg. 

I was curious, so I duplicated the notebook to run the other features. I still believe that weight is an important feature as it relates to mpg, so I then changed my secondary feature. Cylinders, acceleration, and origin had similar performance numbers as displacement. 

However, the pipeline with polynomial features, using weight and model year as my training features had significantly improved scores: training RMSE, 2.91 and R^2, 86.26%; test RMSE, 2.11 and R^2, 85.88%. I would imagine this could be due to advancements in car engineering allowing for a more effective use of fuel, thus having better mpg. The advancements that occurred as model years increased must have a strong correlation to the mpg, allowing this variable to be an important predictive feature.   
